In [2]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

import findspark
findspark.init()
findspark.find()
import pyspark

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)
sqlcontext=SQLContext(sc)

In [3]:
flightDF=spark.read.csv("2015-summary.csv",header=True,inferSchema=True)

In [4]:
flightDF.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [13]:
flightDF.filter(flightDF["DEST_COUNTRY_NAME"]=="United States").groupBy("ORIGIN_COUNTRY_NAME")\
.agg({"count":"sum"}).orderBy("ORIGIN_COUNTRY_NAME",ascending=True).explain()

== Physical Plan ==
*(3) Sort [ORIGIN_COUNTRY_NAME#17 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(ORIGIN_COUNTRY_NAME#17 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#226]
   +- *(2) HashAggregate(keys=[ORIGIN_COUNTRY_NAME#17], functions=[sum(cast(count#18 as bigint))])
      +- Exchange hashpartitioning(ORIGIN_COUNTRY_NAME#17, 200), ENSURE_REQUIREMENTS, [id=#222]
         +- *(1) HashAggregate(keys=[ORIGIN_COUNTRY_NAME#17], functions=[partial_sum(cast(count#18 as bigint))])
            +- *(1) Project [ORIGIN_COUNTRY_NAME#17, count#18]
               +- *(1) Filter (isnotnull(DEST_COUNTRY_NAME#16) AND (DEST_COUNTRY_NAME#16 = United States))
                  +- FileScan csv [DEST_COUNTRY_NAME#16,ORIGIN_COUNTRY_NAME#17,count#18] Batched: false, DataFilters: [isnotnull(DEST_COUNTRY_NAME#16), (DEST_COUNTRY_NAME#16 = United States)], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/Lenovo/PYSPARK/Big Data Course/2015-summary.csv], PartitionFilters: [], PushedFilters:

In [11]:
df=spark.read.csv("flights.csv",header=True,inferSchema=True)

In [7]:
df.orderBy

+----+------+
|Dest| Miles|
+----+------+
| LAX|2402.0|
| LAX|2402.0|
| LAX|2402.0|
| LAX|2402.0|
| LAX|2402.0|
| ORD|1744.0|
| LAX|2402.0|
| LAX|2402.0|
| LAX|2402.0|
+----+------+



In [14]:
df.groupBy("Dest").sum("Miles").withColumnRenamed("sum(Miles)","destination_total").orderBy("destination_total").limit(5)\
.explain()

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#187 ASC NULLS FIRST], output=[Dest#84,destination_total#187])
+- *(2) HashAggregate(keys=[Dest#84], functions=[sum(Miles#85)])
   +- Exchange hashpartitioning(Dest#84, 200), ENSURE_REQUIREMENTS, [id=#264]
      +- *(1) HashAggregate(keys=[Dest#84], functions=[partial_sum(Miles#85)])
         +- FileScan csv [Dest#84,Miles#85] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/Lenovo/PYSPARK/Big Data Course/flights.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Dest:string,Miles:double>


